# NAS Bench 101

jaehwi park <br>
2019.12.22

## 1. Introduction

NAS는 하나의 Optimization Problem으로 활발히 연구되고 있습니다. 하지만 학습이 너무 오래 걸리는 점이 관련 연구의 큰 장애물입니다. 최근 연구들로 그 시간이 많이 줄어들었지만, 여전히 부담스러운 수준입니다. NAS 연구의 또 다른 어려운 점은, 서로 다른 학습 방법 및 다른 탐색 공간에서 정의된 연구 결과들끼리의 비교가 어렵다는 것입니다. 이러한 문제점들을 해소하기 위해 저자들은 NAS-Bench-101을 제안합니다.

NAS-Bench-101은 NAS 연구를 위한 첫 "Architecture-dataset" 입니다. 이를 위해 저자들은 CIFAR-10에서 100 TPU years 이상을 소모해 수 많은 CNN모형들을 만들었다고 합니다.... 그 결과 423k architectures 에 대한 실험 결과를 산출했습니다. 이 정보를 바탕으로 NAS의 전체 탐색 공간에 대한 분석이 이제는 가능해졌다고 주장합니다. 

Contribution을 요약하자면 다음과 같습니다.
 - NAS-Bench-101은 첫 large scale, open-source architecture dataset
 - search space 분석을 위한 dataset 활용법 제시
 - NAS 방법론들 비교를 위한 dataset 활용법 제시

## 2. The NASBench Dataset

본 연구에서는 CIFAR-10을 활용했는데, 그 이유는 CIFAR-10의 크기가 작고, CIFAR-10에서 잘 맞으면 다른 데이터에서도 보통 잘 맞기 때문입니다.

### 2.1 Architectures

다른 연구들처럼 본 연구에서도 Search Space를 제한했습니다. 일반적으로 Cell이라고 부르는 작은 NN 구조를 반복적으로 활용합니다. 기본적인 구조는 Cell을 세 개씩 쌓는 것입니다. 그 후 바로 max-pooling을 하는데 여기에서 width, height를 절반으로 channel은 두 배로 만듭니다. 맨 마지막 Cell 다음에서는 global average pooling을 하고 dense softmax layer를 통과시킵니다. 가장 처음 layer인 *stem*은 3x3 conv & 128 channels입니다. 

<br><center><b> Figure1 Top-Left: 전체구조 </b></center>

![F1_TL](Figure1_TL.png)

Architecture 공간은 DAG로 구성합니다. DAG는 V nodes와 operation을 의미하는 L labels로 이루어집니다. 가장 왼쪽 상단과, 오른쪽 하단은 "IN"과 "OUT"를 의미합니다. 그런데 DAGs는 V와 L이 증가하면 지수적으로 경우의 수가 증가하므로 이를 제어했습니다. 또한, 모든 Conv는 Relu + BatchNorm을 기본으로 합니다.

 - L = 3
   - 3 x 3 Conv.
   - 1 x 1 Conv.
   - 3 x 3 max-pool
 - V <= 7
 - edges <= 9

<br><center><b> Supply1-Figure1: DAG sample </b></center>

![s1f1](S1_Figure1.png)

위와 같은 정의는 ResNet-like 구조와 Inception-like 구조를 모두 포함합니다. 이를 위해  AmoebaNet 등에서 사용되는 separable conv를 안쓰고 일부러 일반 conv를 사용했다고 합니다...

![f1_tr](Figure1_TR.png)

### 2.2 Cell encoding

Cell을 encoding하는 방법은 다양할 수 있는데, 본 논문에서는 7-vertex DAG로 구현하기 위해 7x7 Matrix의 upper-triangular binary matrix로 구현했습니다. 그러므로 이론적인 총 경우의 수는 $2^{21} \times 3^5 = 500M$ unique models 입니다. 하지만 위의 제약 조건에 부합하지 않는 경우 및 중복을 제외하면 총 423k의 unique graphs가 남습니다.

### 2.3 Combine semantics

Graph를 실제 Network로 변환하는 것은 한 가지만 제외하면 간단합니다. 바로 한 개의 Node로 Edges가 몰리는 경우입니다. 이 경우 add 연산 또는 concat 연산이 가능합니다. ResNet& Inception like 구조를 고려하기 위해 output node로 가는 경우에는 concat, 나머지는 sum 연산을 합니다.

### 2.4 Training

학습에는 single, fixed hyperparameters set을 사용했다고 합니다. 이렇게 학습 요건이 주어지면, 알고리즘 간의 비교가 가능합니다.

__Choice of hyperparameters__

50개의 randomly-smapled architectures에서 평균 정도의 정확도를 내는 hyperparameters들을 grid-search 방법으로 찾아냈다고 합니다. 

__Implementation details__

모두 CIFAR-10 Dataset을 이용했고, standard data augmentation(He et al., 2016)을 적용했습니다. 나머지 details는 아래와 같습니다.

![t1](Table1.png)

__3 repeats and 4 epoch budgets__

모든 architectures를 세 번씩 학습하고 평가했습니다. 또한, multi-fidelity optimization methods 평가를 위해 네 가지의 epochs로 학습을 진행했습니다. learning rate는 $E_{stop}$을 기준으로 감소하도록 학습했습니다. 
  - $E_{stop}$ = {4, 12, 36, 108}

따라서 전체 models는 $3 \times 4 \times 423k = 5M$가 됩니다.

### 2.5 Metrics

NASBench Dataset은 (Architecture, $E_{stop}$, trial#) key로 mapping 가능합니다. 포함하고 있는 결과는 다음과 같습니다.
  - training accuracy
  - validation accuracy
  - testing accuracy
  - training time in seconds
  - number of trainable model parameters

### 2.6 Benchmarking methods

NAS의 목표는 $E_{max}$에서 가장 좋은 testing accuracy를 내는 것입니다. 이를 위해 NASBench에서 (A, $E_{stop}$)를 key로 trial#은 random하게 결과를 가져옵니다.

탐색 과정에서, validation accuracy 기준으로 가장 좋은 A를 tracking합니다. 현실 제약조건을 반영하기 위해 fixed limit time까지만 탐색을 진행합니다. 탐색이 끝난 후 A의 test accuracy를 NASBench dataset에서 가져옵니다. 가져온 결과 중 가장 좋은 것과 비교하여 탐색 결과를 평가합니다.

$r(\hat A_i) = f(\hat A_i) - f(A^*)$

상세한 guideline은 다음과 같습니다. (Suppliment 7)

    1. NAS 알고리즘 수행 (본 논문에서는 500 runs)
    2. 

## 3. NASBench as a Dataset

이번 장에서는 NASBench에서 얻은 insights에 대해 정리합니다.

### 3.1 Dataset statistics

대부분의 architecture에서 training set은 100%의 accuracy를 보이고, validation 및 test set에서는 90% 수준의 accuracy를 보였습니다. 가장 좋은 architecture는 test accuracy가 94.32%였고, Resnet-like는 93.12%, Inception-like는 92.95%, 그리고 original ResNet-56은 93.03%의 accuracy를 나타냈다고 합니다.

또한, validation & test accuracy의 correlation을 조사했는데, epochs가 많을 수록 높은 correlation을 보여주었다고 합니다. 이는 strong optimizers가 validation set에 더 overfitting되지 않는다는 것을 의미합니다.

<center><b> Figure 2 </b></center>

![f2](Figure2.png)

<center><b> Figure 3 </b></center>

![f3](Figure3.png)

figure 3 오른쪽 그래프에서 오래 학습한다고 좋은 결과가 나오는 것은 아니라는 점을 확인할 수 있습니다.

### 3.2 Architectural design

<center><b> Figure 4 </b></center>

![f4](Figure4.png)

NASBench는 operation을 변경했을 때의 결과를 비교해볼 수도 있습니다. Figure 4에서 확인할 수 있듯이 Conv 3x3을 다른 연산으로 변경하면 accuracy가 감소합니다.

<center><b> Figure 5 </b></center>

![f5](Figure5.png)

depth는 3에서 가장 좋은 결과를 보이며, width는 5까지 계속 증가합니다.

### 3.3 Locality

NASBench는 비슷한 구조가 비슷한 결과를 나타내는 Locality를 보여줍니다. 여기에서 비슷하다는 의미는 *edit-distance*입니다.

## 4. NASBench as a Benchmark

본 장에서의 목표는 최고의 방법론을 찾는 것이 아닌, "reproducible baseline"의 활용성을 보여주는 것이라고 합니다.

![f7](Figure7.png)

왼쪽 그래프에서 performance, 오른쪽 그래프에서 robustness를 확인할 수 있습니다.